In [142]:
import torch
import torchvision
import torchvision.transforms as transforms
import copy
from cur import cur_decomposition

In [143]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(16 * 4 * 4, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
net = Net()
net.load_state_dict(torch.load("LeNET_300_100_MNIST_Model"))

In [144]:
from torchsummary import summary
device=torch.device("cpu")
model=Net().to(device)
summary(model, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 300]          77,100
            Linear-6                  [-1, 100]          30,100
            Linear-7                   [-1, 10]           1,010
Total params: 110,782
Trainable params: 110,782
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.42
Estimated Total Size (MB): 0.47
----------------------------------------------------------------


In [145]:
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())
    print(param_tensor, "\t", net.state_dict()[param_tensor])

conv1.weight 	 torch.Size([6, 1, 5, 5])
conv1.weight 	 tensor([[[[ 0.1954,  0.0121,  0.1640,  0.1464,  0.2705],
          [ 0.1304, -0.0403,  0.1433,  0.0456, -0.0522],
          [-0.2085, -0.1837, -0.1483, -0.0032, -0.1035],
          [-0.0852, -0.0686,  0.1133, -0.1955, -0.1933],
          [-0.0814, -0.1062, -0.2126, -0.1033,  0.0747]]],


        [[[ 0.1493,  0.1061, -0.1931, -0.0537, -0.1221],
          [ 0.1064, -0.1001, -0.1260, -0.1717, -0.0048],
          [-0.0145, -0.1327, -0.0277, -0.1975,  0.1304],
          [-0.1494, -0.0456, -0.0669, -0.0927, -0.0431],
          [ 0.2032,  0.1092, -0.0361, -0.0539, -0.1006]]],


        [[[ 0.1150, -0.0196, -0.0178, -0.1776, -0.1723],
          [ 0.2551,  0.1801,  0.1082, -0.1142, -0.1189],
          [ 0.0085, -0.0342, -0.0729, -0.1232,  0.0946],
          [ 0.1271,  0.1823,  0.1615,  0.0363,  0.2306],
          [ 0.2858,  0.2635, -0.0239,  0.1917, -0.0294]]],


        [[[ 0.2501,  0.1590,  0.1105,  0.1619, -0.1553],
          [-0.0840,  

In [146]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)

In [147]:
i=0
for parameter in net.parameters():
    i+=1
    print(i," ",parameter.shape,"\n ",parameter,"\n\n")

1   torch.Size([6, 1, 5, 5]) 
  Parameter containing:
tensor([[[[ 0.1954,  0.0121,  0.1640,  0.1464,  0.2705],
          [ 0.1304, -0.0403,  0.1433,  0.0456, -0.0522],
          [-0.2085, -0.1837, -0.1483, -0.0032, -0.1035],
          [-0.0852, -0.0686,  0.1133, -0.1955, -0.1933],
          [-0.0814, -0.1062, -0.2126, -0.1033,  0.0747]]],


        [[[ 0.1493,  0.1061, -0.1931, -0.0537, -0.1221],
          [ 0.1064, -0.1001, -0.1260, -0.1717, -0.0048],
          [-0.0145, -0.1327, -0.0277, -0.1975,  0.1304],
          [-0.1494, -0.0456, -0.0669, -0.0927, -0.0431],
          [ 0.2032,  0.1092, -0.0361, -0.0539, -0.1006]]],


        [[[ 0.1150, -0.0196, -0.0178, -0.1776, -0.1723],
          [ 0.2551,  0.1801,  0.1082, -0.1142, -0.1189],
          [ 0.0085, -0.0342, -0.0729, -0.1232,  0.0946],
          [ 0.1271,  0.1823,  0.1615,  0.0363,  0.2306],
          [ 0.2858,  0.2635, -0.0239,  0.1917, -0.0294]]],


        [[[ 0.2501,  0.1590,  0.1105,  0.1619, -0.1553],
          [-0.0840,  0

In [148]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 89.750000 %


In [149]:
import sys
import numpy as np
import time
from collections import Counter
import random

In [150]:
# import sys
# !{sys.executable} -m pip install cur

In [151]:
# def mysvd(matrix, rank):

# 	"""
# 	INPUT: matrix: user-rating matrix, rank: desired rank.
# 	OUTPUT: returns U, sigma and V_t resulting from the svd decomposition of the matrix.
# 	"""
# 	m = matrix.shape[0]
# 	n = matrix.shape[1]
	
# 	if((rank>m) or (rank>n)):
# 		print("error: rank greater than matrix dimensions.\n")
# 		return;
		
# 	matrix_t = matrix.T
	
# 	A = np.dot(matrix, matrix_t)						#calculate matrix multiplied by its transpose
# 	values1, v1 = np.linalg.eigh(A)						#get eigenvalues and eigenvectors
# 	v1_t = v1.T
# 	v1_t[values1<0] = 0						#discarding negative eigenvalues and corresponding eigenvectors(they are anyway tending to zero)
# 	v1 = v1_t.T
# 	values1[values1<0] = 0
# 	#values1 = np.absolute(values1)
		
# 	values1 = np.sqrt(values1)						#finding singular values.
	
# 	idx = np.argsort(values1)						#sort eigenvalues and eigenvectors in decreasing order
# 	idx = idx[: :-1]
# 	values1 = values1[idx]
# 	v1 = v1[:, idx]
	
# 	U = v1
	
# 	A = np.dot(matrix_t, matrix)						#calculate matrix transpose multiplied by matrix.
# 	values2, v2 = np.linalg.eigh(A)						#get eigenvalues and eigenvectors
# 	#values2 = np.absolute(values2)
# 	v2_t = v2.T
# 	v2_t[values2<0] = 0						#discarding negative eigenvalues and corresponding eigenvectors(they are anyway tending to zero)
# 	v2 = v2_t.T
# 	values2[values2<0] = 0
	
# 	values2 = np.sqrt(values2)						#finding singular values.
	
# 	idx = np.argsort(values2)						#sort eigenvalues and eigenvectors in decreasing order.
# 	idx = idx[: :-1]
# 	values2 = values2[idx]
# 	v2 = v2[:, idx]
	
# 	V = v2
# 	V_t = V.T										#taking V transpose.
	
# 	sigma = np.zeros((m,n))
	
# 	if(m>n):										#setting the dimensions of sigma matrix.
		
# 		sigma[:n, :] = np.diag(values2)
			
# 	elif(n>m):
# 		sigma[:, :m] = np.diag(values1)
		
# 	else:
# 		sigma[:, :] = np.diag(values1)
			
# 	if(m > rank):									#slicing the matrices according to the rank value.
# 		U = U[:, :rank]
# 		sigma = sigma[:rank, :]
	
# 	if(n > rank):
# 		V_t = V_t[:rank, :]
# 		sigma = sigma[:, :rank]
	
# 	check = np.dot(matrix, V_t.T)					
# 	#case = np.divide(check, values2[:rank])
	
# 	s1 = np.sign(check)
# 	s2 = np.sign(U)
# 	c = (s1==s2)
	
# 	for i in range(U.shape[1]):						#choosing the correct signs of eigenvectors in the U matrix.
# 		if(c[0, i]==False):
# 			U[:, i] = U[:, i]*-1
	
	
# 	return U, sigma, V_t

In [152]:
# def mycur(matrix, rank):
	
# 	"""
# 	INPUT: matrix: user-rating matrix, rank: desired rank.
# 	OUTPUT: returns C, U and R resulting from the CUR decomposition of the matrix.
# 	"""
# 	m = matrix.shape[0]
# 	n = matrix.shape[1]
	
# 	if((rank>m) or (rank>n)):
# 		print("error: rank greater than matrix dimensions.\n")
# 		return;
		
# 	C = np.zeros((m, rank))
# 	R = np.zeros((rank, n))
	
# 	matrix_sq = matrix**2
# 	sum_sq = np.sum(matrix_sq)
	
# 	frob_col = np.sum(matrix_sq, axis=0)
# 	probs_col = frob_col/sum_sq				#probability of each column.
	
# 	count=0
# 	count1=0
# 	temp = 0
# 	idx = np.arange(n)						#array of column indexes.
# 	taken_c = []
# 	dup_c = []
	
# 	while(count<rank):
# 		i = np.random.choice(idx, p = probs_col)	#choosing column index based on probability.
# 		count1 = count1+1
# 		if(i not in taken_c):
# 			C[:, count] = matrix[:, i]/np.sqrt(rank*probs_col[i])	#taking column after dividing it with root of rank*probability.
# 			count = count+1
# 			taken_c.append(i)
# 			dup_c.append(1)
# 		else:										#discarding the duplicate column and increasing its count of duplicates.
# 			temp = taken_c.index(i)
# 			dup_c[temp] = dup_c[temp]+1
			
# 	C = np.multiply(C, np.sqrt(dup_c))				#multiply columns by root of number of duplicates.
			
# 	frob_row = np.sum(matrix_sq, axis=1)
# 	probs_row = frob_row/sum_sq					#probability of each row.
	
# 	count=0
# 	count1=0
# 	idx = np.arange(m)							#array of row indexes.
# 	taken_r = []
# 	dup_r = []
	
# 	while(count<rank):
# 		i = np.random.choice(idx, p = probs_row)			#choosing row index based on probability.
# 		count1 = count1+1
# 		if(i not in taken_r):
# 			R[count, :] = matrix[i, :]/np.sqrt(rank*probs_row[i])		#taking row after dividing it with root of rank*probability.
# 			count = count+1
# 			taken_r.append(i)
# 			dup_r.append(1)
# 		else:
# 			temp = taken_r.index(i)							#discarding the duplicate row and increasing its count of duplicates.
# 			dup_r[temp] = dup_r[temp]+1
		
# 	R = np.multiply(R.T, np.sqrt(dup_r))				#multiply rows by root of number of duplicates.
# 	R = R.T
	
# 	W = np.zeros((rank, rank))
	
# 	for i, I in enumerate(taken_r):
# 		for j, J in enumerate(taken_c):				#forming the intersection matrix W.
# 			W[i, j] = matrix[I, J]
	
# 	X, sigma, Y_t = mysvd(W,rank)					#svd decomposition of W.
	
# 	for i in range(rank):
# 		if(sigma[i,i] >= 1):						#taking pseudo-inverse of sigma.
# 			sigma[i,i] = 1/sigma[i,i]
# 		else:
# 			sigma[i,i] = 0
	
# 	U = np.dot(Y_t.T, np.dot(np.dot(sigma,sigma), X.T))		#finding U.
	
# 	return C, U, R

We will first do a CUR decomposition in the first weight layer:

In [153]:
print((net.state_dict()["fc1.weight"]).shape)

torch.Size([300, 256])


So, we will choose a rank less than both the column and row length of this matrix. If the matrix is m*n, the rank should be less than mn/(m+n)  Source: https://github.com/CaptainSame/Singular-Value-Decomposition-and-CUR-matrix-approximation

In [154]:
## rank should be less than 132
# M=net.state_dict()["fc1.weight"].numpy()
# C1,U1,R1=mycur(M, 30)

M=net.state_dict()["fc1.weight"].numpy()
C1, U1, R1 = cur_decomposition(M, 30)

In [155]:
print(type(C1)," ",C1.shape)
print(type(U1)," ",U1.shape)
print(type(R1)," ",R1.shape)

<class 'numpy.ndarray'>   (300, 30)
<class 'numpy.ndarray'>   (30, 30)
<class 'numpy.ndarray'>   (30, 256)


In [156]:
print(U1)

[[ 4.20765923e+28  4.95757757e+13  1.17614836e+14 -1.02659460e+13
   4.84775182e+13 -2.06475482e+29 -1.59641669e+14  1.21624490e+14
   2.06475482e+29  2.26280939e+14 -1.55157717e+13 -1.05186744e+14
  -1.38728803e+14 -5.76827026e+13 -1.58519798e+13 -1.30559884e+14
  -1.73695490e+14 -4.20765923e+28  2.89492498e+13 -1.73695490e+14
  -4.52332031e+13  2.15051563e+14  1.11661390e+14 -1.27428391e+14
  -1.64907333e+14 -1.46814045e+14 -2.44259286e+14  1.15796999e+14
  -1.81966708e+14  7.23731266e+13]
 [-8.79164358e+32 -3.24777078e+17  5.68656178e+17 -5.17929140e+15
   1.32263819e+17 -1.78953111e+32 -2.99976115e+17  1.00657824e+17
   1.78953111e+32  3.34984050e+17 -3.86483663e+17  7.54116714e+16
  -6.50888550e+17 -3.66125990e+16  6.79559175e+16  1.53462886e+16
  -1.28524842e+18  8.79164358e+32  2.81707077e+17 -1.05483168e+17
   5.06286175e+17  7.39347009e+17  3.51092252e+16  1.85011437e+17
  -7.02747660e+16 -7.04195966e+17 -3.78345902e+17  2.81623359e+17
  -1.75908941e+17  4.57749609e+17]
 [ 5.9

In [157]:
# for i in range(len(U1)):
#     for j in range(len(U1[0])):
#         if(U1[i][j]!=0):
#             print(U1[i][j])

In [158]:
# for i in range(len(U2)):
#     for j in range(len(U2[0])):
#         if(U2[i][j]!=0.0):
#             print(U2[i][j])

Will do the same for the next layer

In [159]:
print((net.state_dict()["fc2.weight"]).shape)

torch.Size([100, 300])


In [160]:
## rank should be less than 75
# C2,U2,R2=mycur((net.state_dict()["fc2.weight"]), 75)

M=net.state_dict()["fc2.weight"].numpy()
C2, U2, R2 = cur_decomposition(M, 10)

In [161]:
print(type(C2)," ",C2.shape)
print(type(U2)," ",U2.shape)
print(type(R2)," ",R2.shape)

<class 'numpy.ndarray'>   (100, 10)
<class 'numpy.ndarray'>   (10, 10)
<class 'numpy.ndarray'>   (10, 300)


So now we have the CUR decompositions, the we have to set the new network architectures accordingly, and we will lock the weight according the received weights, we will though have to learn the bias for the intermediate layer

In [162]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(16 * 4 * 4, C1.shape[1])
        self.fc2 = nn.Linear(C1.shape[1], C1.shape[1])
        self.fc3 = nn.Linear(C1.shape[1], 300)
        self.fc4 = nn.Linear(300, C2.shape[1])
        self.fc5 = nn.Linear(C2.shape[1], C2.shape[1])
        self.fc6 = nn.Linear(C2.shape[1], 100)
        self.fc7 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return F.log_softmax(x, dim=1)
    
net2 = Net()

In [163]:
for param_tensor in net2.state_dict():
    print(param_tensor, "\t", net2.state_dict()[param_tensor].size())
    print(param_tensor, "\t", net2.state_dict()[param_tensor])

conv1.weight 	 torch.Size([6, 1, 5, 5])
conv1.weight 	 tensor([[[[ 0.0900,  0.1536, -0.1308, -0.0516,  0.0188],
          [-0.0993,  0.0011, -0.0562,  0.0426,  0.0112],
          [-0.0561, -0.1343,  0.0532,  0.0120, -0.0968],
          [ 0.1203, -0.1473,  0.1544, -0.0839, -0.1125],
          [ 0.0442, -0.0685, -0.1605,  0.0326, -0.1899]]],


        [[[ 0.1672,  0.0258,  0.1177, -0.1778, -0.1845],
          [-0.1936,  0.1283, -0.0360, -0.1529, -0.1362],
          [ 0.0231,  0.1961, -0.0592,  0.1808,  0.0369],
          [-0.0354,  0.1017, -0.1690,  0.1878, -0.0022],
          [-0.1053, -0.0371, -0.1017, -0.1258,  0.1537]]],


        [[[ 0.1811,  0.0007, -0.0256,  0.0512, -0.1907],
          [ 0.1285, -0.1656, -0.1578, -0.0779,  0.1327],
          [ 0.1014, -0.0940, -0.0349,  0.0712, -0.1807],
          [-0.1625,  0.0367, -0.0186,  0.0747,  0.0333],
          [ 0.0391, -0.1088,  0.0828, -0.0922, -0.1728]]],


        [[[-0.0269, -0.0410,  0.0815,  0.1867, -0.1617],
          [-0.0192, -

First 2 conv layers we will copy from the old network

In [164]:
print((net.state_dict()["conv1.weight"]).shape)
# print(net.state_dict()["conv1.weight"])
for i in range((net.state_dict()["conv1.weight"]).shape[0]):
    for j in range((net.state_dict()["conv1.weight"]).shape[1]):
        for k in range((net.state_dict()["conv1.weight"]).shape[2]):
            for l in range((net.state_dict()["conv1.weight"]).shape[3]):
                net2.state_dict()["conv1.weight"][i][j][k][l]=net.state_dict()["conv1.weight"][i][j][k][l]
                
# print(net2.state_dict()["conv1.weight"])
print((net2.state_dict()["conv1.weight"]).shape)

torch.Size([6, 1, 5, 5])
torch.Size([6, 1, 5, 5])


In [165]:
print((net.state_dict()["conv1.bias"]).shape)
print(net.state_dict()["conv1.bias"])
for i in range((net.state_dict()["conv1.bias"]).shape[0]):
    net2.state_dict()["conv1.bias"][i]=net.state_dict()["conv1.bias"][i]
                
print(net2.state_dict()["conv1.bias"])
print((net2.state_dict()["conv1.bias"]).shape)

torch.Size([6])
tensor([0.0889, 0.2020, 0.0691, 0.1976, 0.1579, 0.0756])
tensor([0.0889, 0.2020, 0.0691, 0.1976, 0.1579, 0.0756])
torch.Size([6])


In [166]:
print((net.state_dict()["conv2.weight"]).shape)
print(net.state_dict()["conv2.weight"])
for i in range((net.state_dict()["conv2.weight"]).shape[0]):
    for j in range((net.state_dict()["conv2.weight"]).shape[1]):
        for k in range((net.state_dict()["conv2.weight"]).shape[2]):
            for l in range((net.state_dict()["conv2.weight"]).shape[3]):
                net2.state_dict()["conv2.weight"][i][j][k][l]=net.state_dict()["conv2.weight"][i][j][k][l]
                
print(net2.state_dict()["conv2.weight"])
print((net2.state_dict()["conv2.weight"]).shape)

torch.Size([16, 6, 5, 5])
tensor([[[[-0.0274,  0.0668,  0.0488,  0.0398, -0.0665],
          [-0.0608,  0.0753, -0.0379, -0.0708, -0.0499],
          [ 0.0213, -0.0393,  0.0602,  0.0631, -0.0172],
          [-0.0192, -0.0213,  0.0593,  0.0295, -0.0479],
          [ 0.0122,  0.0283, -0.0171, -0.0418,  0.0804]],

         [[ 0.0871, -0.0707,  0.0819,  0.0117,  0.0575],
          [-0.0282,  0.0007,  0.0180, -0.0617, -0.0293],
          [ 0.0198,  0.0782,  0.0392, -0.0338, -0.0277],
          [ 0.0431, -0.0100,  0.0603, -0.0399,  0.0026],
          [-0.0332,  0.0662,  0.0012,  0.0095, -0.0582]],

         [[ 0.0561,  0.0449,  0.0354, -0.0410,  0.0272],
          [ 0.0724,  0.0942, -0.0040,  0.0277,  0.0081],
          [ 0.0270, -0.0307, -0.0064, -0.0385,  0.0159],
          [ 0.0504,  0.0809,  0.0430,  0.0777,  0.0367],
          [-0.0646,  0.0218, -0.0101,  0.0375,  0.0262]],

         [[-0.0235, -0.0858,  0.0660, -0.0673, -0.0697],
          [ 0.0078,  0.0719, -0.0181, -0.0195,  0.0016],

In [167]:
print((net.state_dict()["conv2.bias"]).shape)
print(net.state_dict()["conv2.bias"])
for i in range((net.state_dict()["conv2.bias"]).shape[0]):
    net2.state_dict()["conv2.bias"][i]=net.state_dict()["conv2.bias"][i]
                
print(net2.state_dict()["conv2.bias"])
print((net2.state_dict()["conv2.bias"]).shape)

torch.Size([16])
tensor([ 0.0678, -0.0424, -0.0553, -0.0554, -0.0523, -0.0426,  0.0439, -0.0724,
         0.0329, -0.0029,  0.0769,  0.0866,  0.0797, -0.0330, -0.0008, -0.0292])
tensor([ 0.0678, -0.0424, -0.0553, -0.0554, -0.0523, -0.0426,  0.0439, -0.0724,
         0.0329, -0.0029,  0.0769,  0.0866,  0.0797, -0.0330, -0.0008, -0.0292])
torch.Size([16])


Now, we will copy R1 to fc1, U1 to fc2, and C1 to fc3 weights, we will copy the bias of fc1 of old network to fc3 of new network, and fc2 of new model biases will be kept as 0. 

In [168]:
print(R1.shape)
print(type(net2.state_dict()["fc1.weight"]), " ", net2.state_dict()["fc1.weight"].shape)

(30, 256)
<class 'torch.Tensor'>   torch.Size([30, 256])


In [169]:
R1_t=torch.from_numpy(R1)
print(type(R1_t), " ", R1_t.shape)

<class 'torch.Tensor'>   torch.Size([30, 256])


In [170]:
print(net2.state_dict()["fc1.weight"])
for i in range((net2.state_dict()["fc1.weight"]).shape[0]):
    for j in range((net2.state_dict()["fc1.weight"]).shape[1]):
        net2.state_dict()["fc1.weight"][i][j]=R1_t[i][j]
                
print(net2.state_dict()["fc1.weight"])

tensor([[-0.0623,  0.0125, -0.0218,  ..., -0.0153, -0.0328,  0.0237],
        [-0.0517, -0.0239,  0.0306,  ..., -0.0081,  0.0016,  0.0079],
        [-0.0043, -0.0162, -0.0175,  ...,  0.0530, -0.0440, -0.0591],
        ...,
        [ 0.0132,  0.0558, -0.0354,  ...,  0.0342, -0.0308,  0.0428],
        [ 0.0588,  0.0101,  0.0165,  ..., -0.0292,  0.0526, -0.0556],
        [ 0.0463,  0.0167,  0.0105,  ...,  0.0556, -0.0072,  0.0018]])
tensor([[ 0.2035,  0.0628, -0.1114,  ...,  0.1423,  0.0183,  0.1204],
        [ 0.0804, -0.0442, -0.1846,  ...,  0.1065,  0.1225, -0.0888],
        [-0.1880, -0.0530, -0.0524,  ...,  0.1186,  0.0864,  0.1761],
        ...,
        [ 0.0383,  0.1524, -0.0951,  ...,  0.0868,  0.0490, -0.1920],
        [ 0.1769,  0.1474, -0.0726,  ...,  0.0688, -0.1075,  0.0550],
        [-0.1786, -0.0219,  0.1262,  ...,  0.0131,  0.1254,  0.1222]])


In [171]:
# print(net2.state_dict()["fc1.bias"].shape)
# for i in range(net2.state_dict()["fc1.bias"].shape[0]):
#     net2.state_dict()["fc1.bias"][i]=0.0
# print((net2.state_dict()["fc1.bias"]))

In [172]:
print(U1.shape)
print(type(net2.state_dict()["fc2.weight"]), " ", net2.state_dict()["fc2.weight"].shape)

(30, 30)
<class 'torch.Tensor'>   torch.Size([30, 30])


In [173]:
U1_t=torch.from_numpy(U1)
print(type(U1_t), " ", U1_t.shape)

<class 'torch.Tensor'>   torch.Size([30, 30])


In [174]:
# print(net2.state_dict()["fc2.weight"])
# for i in range((net2.state_dict()["fc2.weight"]).shape[0]):
#     for j in range((net2.state_dict()["fc2.weight"]).shape[1]):
#         net2.state_dict()["fc2.weight"][i][j]=U1_t[i][j]
                
# print(net2.state_dict()["fc2.weight"])

In [175]:
# print(net2.state_dict()["fc2.bias"].shape)
# for i in range(net2.state_dict()["fc2.bias"].shape[0]):
#     net2.state_dict()["fc2.bias"][i]=0.0
# print((net2.state_dict()["fc2.bias"]))

In [176]:
print(C1.shape)
print(type(net2.state_dict()["fc3.weight"]), " ", net2.state_dict()["fc3.weight"].shape)

(300, 30)
<class 'torch.Tensor'>   torch.Size([300, 30])


In [177]:
C1_t=torch.from_numpy(C1)
print(type(C1_t), " ", C1_t.shape)

<class 'torch.Tensor'>   torch.Size([300, 30])


In [178]:
print(net2.state_dict()["fc3.weight"])
for i in range((net2.state_dict()["fc3.weight"]).shape[0]):
    for j in range((net2.state_dict()["fc3.weight"]).shape[1]):
        net2.state_dict()["fc3.weight"][i][j]=C1_t[i][j]
                
print(net2.state_dict()["fc3.weight"])

tensor([[-0.0269, -0.0573,  0.0506,  ..., -0.1066, -0.1333,  0.0995],
        [ 0.0331, -0.0010,  0.0157,  ...,  0.0647, -0.1822,  0.0523],
        [ 0.1466, -0.0339, -0.1775,  ...,  0.0089,  0.1290,  0.0570],
        ...,
        [-0.1750, -0.1214,  0.0075,  ..., -0.0690,  0.1016, -0.1114],
        [ 0.0595, -0.1680, -0.1121,  ...,  0.0413, -0.1750, -0.1459],
        [ 0.1024,  0.1502,  0.1083,  ...,  0.0736,  0.0417, -0.1492]])
tensor([[ 0.1696,  0.2068,  0.0329,  ..., -0.0767, -0.1408, -0.0829],
        [ 0.1813, -0.0300, -0.1123,  ..., -0.0377,  0.1526,  0.1720],
        [ 0.1810,  0.2029, -0.1120,  ..., -0.0022,  0.1157, -0.1247],
        ...,
        [-0.1048,  0.0124, -0.0223,  ..., -0.0357, -0.0016, -0.1841],
        [ 0.0530,  0.0888,  0.0624,  ...,  0.1580, -0.0647, -0.0031],
        [ 0.1075,  0.0063, -0.1490,  ..., -0.0708, -0.1324,  0.0591]])


In [179]:
print(net2.state_dict()["fc3.bias"].shape)
for i in range(net2.state_dict()["fc3.bias"].shape[0]):
    net2.state_dict()["fc3.bias"][i]=net.state_dict()["fc1.bias"][i]
print((net2.state_dict()["fc3.bias"]))

torch.Size([300])
tensor([-0.0613, -0.0096,  0.0004,  0.0227, -0.0270,  0.0271, -0.0524, -0.0336,
         0.0488, -0.0171,  0.0535, -0.0261,  0.0042,  0.0027, -0.0281,  0.0008,
        -0.0528, -0.0025,  0.0006, -0.0046, -0.0006,  0.0142, -0.0409,  0.0599,
         0.0513, -0.0374,  0.0186,  0.0505,  0.0519, -0.0291, -0.0098,  0.0555,
        -0.0124, -0.0360,  0.0028,  0.0044, -0.0460,  0.0139, -0.0303,  0.0069,
        -0.0449, -0.0036,  0.0203,  0.0664,  0.0219, -0.0148,  0.0184,  0.0326,
         0.0358, -0.0465, -0.0083, -0.0321, -0.0350, -0.0389, -0.0196,  0.0551,
        -0.0545,  0.0617, -0.0530,  0.0079, -0.0598,  0.0029, -0.0224,  0.0247,
        -0.0024, -0.0387, -0.0484, -0.0210, -0.0130, -0.0341,  0.0596, -0.0330,
        -0.0460, -0.0284, -0.0112, -0.0368, -0.0115, -0.0361, -0.0175,  0.0584,
         0.0427,  0.0172, -0.0470,  0.0225,  0.0001, -0.0539, -0.0198, -0.0206,
        -0.0474,  0.0300, -0.0206,  0.0304,  0.0077,  0.0543,  0.0369, -0.0293,
        -0.0006, -0.05

In [180]:
print(R2.shape)
print(type(net2.state_dict()["fc4.weight"]), " ", net2.state_dict()["fc4.weight"].shape)

(10, 300)
<class 'torch.Tensor'>   torch.Size([10, 300])


In [181]:
R2_t=torch.from_numpy(R2)
print(type(R2_t), " ", R2_t.shape)

<class 'torch.Tensor'>   torch.Size([10, 300])


In [182]:
print(net2.state_dict()["fc4.weight"])
for i in range((net2.state_dict()["fc4.weight"]).shape[0]):
    for j in range((net2.state_dict()["fc4.weight"]).shape[1]):
        net2.state_dict()["fc4.weight"][i][j]=R2_t[i][j]
                
print(net2.state_dict()["fc4.weight"])

tensor([[-0.0191, -0.0282, -0.0246,  ...,  0.0153, -0.0391,  0.0268],
        [-0.0247,  0.0006, -0.0227,  ...,  0.0561, -0.0460,  0.0495],
        [ 0.0163, -0.0164,  0.0125,  ...,  0.0111, -0.0424, -0.0210],
        ...,
        [-0.0265,  0.0267,  0.0349,  ...,  0.0219,  0.0213, -0.0434],
        [ 0.0278, -0.0463,  0.0232,  ..., -0.0158, -0.0243,  0.0320],
        [-0.0518,  0.0449, -0.0575,  ...,  0.0402, -0.0471,  0.0121]])
tensor([[-0.1567, -0.1528,  0.1927,  ...,  0.1377,  0.0056, -0.1173],
        [ 0.1560,  0.1992, -0.1633,  ...,  0.0552, -0.1572, -0.1808],
        [ 0.0742,  0.0217,  0.0274,  ..., -0.0117,  0.1895, -0.0903],
        ...,
        [ 0.0335,  0.1006,  0.0834,  ..., -0.0087,  0.0524, -0.1107],
        [-0.1276,  0.0518,  0.1554,  ..., -0.0376, -0.1273, -0.1761],
        [ 0.0742,  0.0217,  0.0274,  ..., -0.0117,  0.1895, -0.0903]])


In [183]:
# print(net2.state_dict()["fc4.bias"].shape)
# for i in range(net2.state_dict()["fc4.bias"].shape[0]):
#     net2.state_dict()["fc4.bias"][i]=0.0
# print((net2.state_dict()["fc4.bias"]))

In [184]:
print(U2.shape)
print(type(net2.state_dict()["fc5.weight"]), " ", net2.state_dict()["fc5.weight"].shape)

(10, 10)
<class 'torch.Tensor'>   torch.Size([10, 10])


In [185]:
U2_t=torch.from_numpy(U2)
print(type(U2_t), " ", U2_t.shape)

<class 'torch.Tensor'>   torch.Size([10, 10])


In [186]:
# print(net2.state_dict()["fc5.weight"])
# for i in range((net2.state_dict()["fc5.weight"]).shape[0]):
#     for j in range((net2.state_dict()["fc5.weight"]).shape[1]):
#         net2.state_dict()["fc5.weight"][i][j]=U2_t[i][j]
                
# print(net2.state_dict()["fc5.weight"])

In [187]:
# print(net2.state_dict()["fc5.bias"].shape)
# for i in range(net2.state_dict()["fc5.bias"].shape[0]):
#     net2.state_dict()["fc5.bias"][i]=0.0
# print((net2.state_dict()["fc5.bias"]))

In [188]:
print(C2.shape)
print(type(net2.state_dict()["fc6.weight"]), " ", net2.state_dict()["fc6.weight"].shape)

(100, 10)
<class 'torch.Tensor'>   torch.Size([100, 10])


In [189]:
C2_t=torch.from_numpy(C2)
print(type(C2_t), " ", C2_t.shape)

<class 'torch.Tensor'>   torch.Size([100, 10])


In [190]:
print(net2.state_dict()["fc6.weight"])
for i in range((net2.state_dict()["fc6.weight"]).shape[0]):
    for j in range((net2.state_dict()["fc6.weight"]).shape[1]):
        net2.state_dict()["fc6.weight"][i][j]=C2_t[i][j]
                
print(net2.state_dict()["fc6.weight"])

tensor([[ 0.0346, -0.0153,  0.2567,  0.2164,  0.2137, -0.0128,  0.0750,  0.2415,
          0.1514, -0.1079],
        [ 0.1074,  0.0682,  0.0755,  0.2846, -0.1891,  0.0399, -0.3014, -0.1743,
          0.2138, -0.1211],
        [ 0.1390,  0.1714,  0.2788, -0.1132, -0.0616, -0.2858, -0.2834, -0.0342,
          0.1886, -0.0804],
        [ 0.2082,  0.1098, -0.2608, -0.2154,  0.0940,  0.2232,  0.0460,  0.2574,
         -0.2038, -0.1220],
        [-0.1992, -0.1435,  0.0298,  0.1491, -0.0362,  0.1614, -0.2138,  0.0467,
          0.1552, -0.0780],
        [ 0.0805, -0.0321, -0.0939, -0.0552,  0.2073,  0.1992,  0.0954, -0.0303,
          0.2180,  0.0504],
        [-0.2889,  0.2946,  0.3054, -0.0737, -0.1886, -0.0870,  0.1536,  0.0287,
          0.2395,  0.1931],
        [-0.2142,  0.0142,  0.3008,  0.2463,  0.1960,  0.2569,  0.0135,  0.0464,
         -0.2759, -0.0670],
        [ 0.2926,  0.1460,  0.0017,  0.1442, -0.0220,  0.2843, -0.0397,  0.2799,
         -0.0571, -0.2632],
        [-0.0019, -

In [191]:
print(net2.state_dict()["fc6.bias"].shape)
for i in range(net2.state_dict()["fc6.bias"].shape[0]):
    net2.state_dict()["fc6.bias"][i]=net.state_dict()["fc2.bias"][i]
print((net2.state_dict()["fc6.bias"]))

torch.Size([100])
tensor([-0.0478,  0.0164,  0.0575, -0.0453,  0.0055, -0.0306, -0.0436, -0.0382,
        -0.0327, -0.0289, -0.0538,  0.0604, -0.0405, -0.0221,  0.0478, -0.0396,
         0.0512,  0.0441, -0.0344, -0.0511,  0.0191,  0.0464,  0.0358, -0.0286,
         0.0001,  0.0551, -0.0305,  0.0320, -0.0183,  0.0414, -0.0293, -0.0355,
         0.0163,  0.0311, -0.0088,  0.0357,  0.0183,  0.0204, -0.0518, -0.0276,
         0.0451,  0.0416, -0.0216,  0.0268,  0.0062,  0.0445, -0.0164,  0.0325,
         0.0003,  0.0565, -0.0114, -0.0467,  0.0377, -0.0437, -0.0243,  0.0100,
        -0.0268,  0.0646,  0.0341,  0.0212, -0.0423,  0.0461, -0.0142,  0.0245,
         0.0349,  0.0400, -0.0025, -0.0210,  0.0286,  0.0610, -0.0223,  0.0475,
        -0.0250,  0.0444, -0.0117, -0.0377,  0.0560, -0.0239, -0.0067, -0.0554,
        -0.0620,  0.0282,  0.0504, -0.0132,  0.0691, -0.0250,  0.0534,  0.0594,
         0.0506, -0.0099,  0.0007,  0.0260, -0.0104, -0.0088,  0.0347,  0.0226,
        -0.0221, -0.01

In [192]:
print(net2.state_dict()["fc7.weight"])
for i in range((net2.state_dict()["fc7.weight"]).shape[0]):
    for j in range((net2.state_dict()["fc7.weight"]).shape[1]):
        net2.state_dict()["fc7.weight"][i][j]=net.state_dict()["fc3.weight"][i][j]
                
print(net2.state_dict()["fc7.weight"])

tensor([[ 0.0602,  0.0841,  0.0457,  0.0804, -0.0555,  0.0306,  0.0438, -0.0871,
          0.0006, -0.0890, -0.0535,  0.0292,  0.0177,  0.0538,  0.0464, -0.0426,
          0.0378,  0.0577, -0.0636,  0.0548,  0.0825, -0.0481,  0.0431, -0.0199,
         -0.0796, -0.0688,  0.0235,  0.0420, -0.0255, -0.0010, -0.0877, -0.0065,
          0.0562, -0.0541,  0.0307,  0.0743,  0.0085, -0.0821,  0.0146, -0.0388,
         -0.0213, -0.0517,  0.0537,  0.0071, -0.0279,  0.0362, -0.0869, -0.0243,
          0.0747,  0.0768, -0.0632,  0.0550,  0.0046, -0.0705,  0.0343, -0.0658,
         -0.0742,  0.0027, -0.0820,  0.0884, -0.0940, -0.0812,  0.0764,  0.0008,
          0.0591,  0.0858, -0.0021, -0.0253, -0.0342, -0.0464,  0.0335,  0.0429,
          0.0615,  0.0555, -0.0239,  0.0870, -0.0090, -0.0993,  0.0882, -0.0212,
         -0.0995,  0.0380,  0.0272,  0.0416, -0.0976, -0.0713,  0.0346, -0.0415,
          0.0499, -0.0548, -0.0202, -0.0638,  0.0946, -0.0506, -0.0301,  0.0326,
         -0.0865,  0.0537, -

In [193]:
print(net2.state_dict()["fc7.bias"].shape)
for i in range(net2.state_dict()["fc7.bias"].shape[0]):
    net2.state_dict()["fc7.bias"][i]=net.state_dict()["fc3.bias"][i]
print((net2.state_dict()["fc7.bias"]))

torch.Size([10])
tensor([ 0.0504,  0.1393, -0.0627, -0.0188,  0.0212, -0.0102, -0.0827, -0.0165,
        -0.1006, -0.0129])


In [194]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 10.100000 %


Here the accuracy actually come up to 76.5, the actual accuracy without any pruning was 89.7. Let's try to freeze the previous layers, and do a fine_tuning. 

In [195]:
i=0
for parameter in net2.parameters():
    i+=1
    if(i<5):
        parameter.requires_grad=False
    elif(i==5 or i==9 or i==11 or i==15):
        parameter.requires_grad=False
    else:
        parameter.requires_grad=True

In [196]:
i=0
for parameter in net2.parameters():
    i+=1
    print(i," ",parameter.requires_grad)

1   False
2   False
3   False
4   False
5   False
6   True
7   True
8   True
9   False
10   True
11   False
12   True
13   True
14   True
15   False
16   True
17   True
18   True


In [197]:
i=0
for parameter in net2.parameters():
    i+=1
    print(i," ",parameter.shape,"\n ",parameter,"\n\n")

1   torch.Size([6, 1, 5, 5]) 
  Parameter containing:
tensor([[[[ 0.1954,  0.0121,  0.1640,  0.1464,  0.2705],
          [ 0.1304, -0.0403,  0.1433,  0.0456, -0.0522],
          [-0.2085, -0.1837, -0.1483, -0.0032, -0.1035],
          [-0.0852, -0.0686,  0.1133, -0.1955, -0.1933],
          [-0.0814, -0.1062, -0.2126, -0.1033,  0.0747]]],


        [[[ 0.1493,  0.1061, -0.1931, -0.0537, -0.1221],
          [ 0.1064, -0.1001, -0.1260, -0.1717, -0.0048],
          [-0.0145, -0.1327, -0.0277, -0.1975,  0.1304],
          [-0.1494, -0.0456, -0.0669, -0.0927, -0.0431],
          [ 0.2032,  0.1092, -0.0361, -0.0539, -0.1006]]],


        [[[ 0.1150, -0.0196, -0.0178, -0.1776, -0.1723],
          [ 0.2551,  0.1801,  0.1082, -0.1142, -0.1189],
          [ 0.0085, -0.0342, -0.0729, -0.1232,  0.0946],
          [ 0.1271,  0.1823,  0.1615,  0.0363,  0.2306],
          [ 0.2858,  0.2635, -0.0239,  0.1917, -0.0294]]],


        [[[ 0.2501,  0.1590,  0.1105,  0.1619, -0.1553],
          [-0.0840,  0

In [198]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

In [199]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=0.001, momentum=0.9)

In [200]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print("[{}, {}] loss: {}".format
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Retraining')

[1, 20] loss: 0.02369382381439209
[1, 40] loss: 0.022484357953071594
[1, 60] loss: 0.021835370898246765
[1, 80] loss: 0.021332664966583252
[1, 100] loss: 0.020633640050888063
[1, 120] loss: 0.019891524612903597
[1, 140] loss: 0.019000263094902037
[1, 160] loss: 0.018381307542324067
[1, 180] loss: 0.01752492183446884
[1, 200] loss: 0.016700281143188476
[1, 220] loss: 0.016073706805706023
[1, 240] loss: 0.015138331830501556
[1, 260] loss: 0.014765093982219696
[1, 280] loss: 0.013840872466564179
[1, 300] loss: 0.01340965086221695
[2, 20] loss: 0.012886070609092713
[2, 40] loss: 0.012072327196598053
[2, 60] loss: 0.011500070571899414
[2, 80] loss: 0.011297782838344574
[2, 100] loss: 0.010769688069820404
[2, 120] loss: 0.010553561955690384
[2, 140] loss: 0.010287893176078797
[2, 160] loss: 0.009735599726438521
[2, 180] loss: 0.009486865878105164
[2, 200] loss: 0.009203378647565841
[2, 220] loss: 0.009415340542793273
[2, 240] loss: 0.009040082484483718
[2, 260] loss: 0.00876192718744278
[2, 

In [201]:
torch.save(net2.state_dict(), "LeNET_x_y_MNIST_Model_My_Exiperiment_3CUR_Fine_Tuned")

In [202]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(16 * 4 * 4, len(net.state_dict()["fc1.weight"]))
#         self.fc2 = nn.Linear(len(net.state_dict()["fc1.weight"]), len(net.state_dict()["fc2.weight"]))
#         self.fc3 = nn.Linear(len(net.state_dict()["fc2.weight"]), 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 4 * 4)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return F.log_softmax(x, dim=1)
    
# net = Net()
# net.load_state_dict(torch.load("LeNET_x_y_MNIST_Model_My_Exiperiment_2_Fine_Tuned"))

In [203]:
from torchsummary import summary
device=torch.device("cpu")
model=Net().to(device)
summary(model, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                   [-1, 30]           7,710
            Linear-6                   [-1, 30]             930
            Linear-7                  [-1, 300]           9,300
            Linear-8                   [-1, 10]           3,010
            Linear-9                   [-1, 10]             110
           Linear-10                  [-1, 100]           1,100
           Linear-11                   [-1, 10]           1,010
Total params: 25,742
Trainable params: 25,742
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

So, number of parametes reduced down to 86k from 110 k

In [204]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 85.440000 %


And the accuracy improved from 89.7 to 92.94

Resources:
[https://towardsdatascience.com/how-to-cluster-in-high-dimensions-4ef693bacc6] [1/11/2019]